### Tools
Models can request to call tools that perform tasks such as fetching data from a database, searching the web, or running code. Tools are pairings of:
1. A schema, including the name of the tool, a description, and/or argument definitions (often a JSON schema)
2. A function or coroutine to execute.

In [1]:
import os
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

model = init_chat_model("groq:qwen/qwen3-32b")
response = model.invoke("Why do parrots talk?")
response

/Users/huzi/Desktop/RAG/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AIMessage(content='<think>\nOkay, so the user is asking why parrots talk. Let me start by recalling what I know about parrots. Parrots are known for their ability to mimic human speech, but why do they do that? I remember reading that it\'s related to their social behavior. Maybe they imitate to communicate with their flock? But how does that translate to talking to humans?\n\nFirst, I should confirm if all parrots can talk. I think only certain species are better at it, like African Greys or Amazon parrots. Maybe their vocal apparatus allows them to mimic sounds. Their syrinx, which is their vocal organ, is probably more complex than other birds. That might give them the physical capability to produce a range of sounds.\n\nThen there\'s the social aspect. Parrots are highly social animals. In the wild, they live in flocks and need to communicate. Mimicking sounds could be a way to bond with others in the flock. If they\'re kept as pets, they might mimic humans to bond with their owner

In [2]:
from langchain.tools import tool

@tool
def get_weather(location:str)->str:
    """Get the weather at a location"""
    return f"It's sunny in {location}"


model_with_tools=model.bind_tools([get_weather])

In [3]:
response = model_with_tools.invoke("What's the weather like in Boston?")
print(response)
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

content='' additional_kwargs={'reasoning_content': "Okay, the user is asking about the weather in Boston. I need to use the get_weather function. Let me check the function parameters. It requires a location, which is Boston here. I'll call the function with location set to Boston.\n", 'tool_calls': [{'id': '943dd2qmk', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 154, 'total_tokens': 227, 'completion_time': 0.110563006, 'completion_tokens_details': {'reasoning_tokens': 49}, 'prompt_time': 0.007590486, 'prompt_tokens_details': None, 'queue_time': 0.048142761, 'total_time': 0.118153492}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019c3cda-f9ae-7203-95f9-5ad6b641fdd6-0' tool_calls=[{'name': 'get_weather', 'args': {'locat

### Tool Execution Loops

In [4]:
# Step 1: Model generates tool calls
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)
# "The current weather in Boston is 72°F and sunny."

The weather in Boston is currently sunny. A perfect day to enjoy outdoor activities! 🌞


In [5]:
messages

[{'role': 'user', 'content': "What's the weather in Boston?"},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the weather in Boston. I need to use the get_weather function. The function requires a location parameter. Boston is the location here. So I should call get_weather with location set to "Boston". Let me make sure there\'s no typo. Everything looks good. I\'ll format the tool call as specified.\n', 'tool_calls': [{'id': '20qs62500', 'function': {'arguments': '{"location":"Boston"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 153, 'total_tokens': 246, 'completion_time': 0.153379107, 'completion_tokens_details': {'reasoning_tokens': 69}, 'prompt_time': 0.006318546, 'prompt_tokens_details': None, 'queue_time': 0.046923754, 'total_time': 0.159697653}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_